In [1]:
import fix_yahoo_finance as yf  
 


In [2]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import matplotlib.pyplot as plt

def max_sharpe_port(stocks, start, end):
    #download daily price data for each of the stocks in the portfolio
    #data = web.DataReader(stocks,data_source='yahoo',start='01/01/2010')['Adj Close']
    try:
        data = yf.download(stocks,start,end)['Adj Close']
    except:
        return max_sharpe_port(stocks)
    data.sort_index(inplace=True)
 
    #convert daily stock prices into daily returns
    returns = data.pct_change()
 
    #calculate mean daily return and covariance of daily returns
    mean_daily_returns = returns.mean()
    cov_matrix = returns.cov()
 
    #set number of runs of random portfolio weights
    num_portfolios = 50000
 
    #set up array to hold results
    #We have increased the size of the array to hold the weight values for each stock
    results = np.zeros((4+len(stocks)-1,num_portfolios))
 
    for i in range(num_portfolios):
        #select random weights for portfolio holdings
        weights = np.array(np.random.random(len(stocks)))
        #rebalance weights to sum to 1
        weights /= np.sum(weights)
    
        #calculate portfolio return and volatility
        portfolio_return = np.sum(mean_daily_returns * weights) * 252
        portfolio_std_dev = np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) * np.sqrt(252)
    
        #store results in results array
        results[0,i] = portfolio_return
        results[1,i] = portfolio_std_dev
        #store Sharpe Ratio (return / volatility) - risk free rate element excluded for simplicity
        results[2,i] = results[0,i] / results[1,i]
        #iterate through the weight vector and add data to results array
        for j in range(len(weights)):
            results[j+3,i] = weights[j]
     
    #convert results array to Pandas DataFrame
    columns = ['ret', 'stdev', 'sharpe'] + stocks
    results_frame = pd.DataFrame(results.T,columns= columns)
 
    #locate position of portfolio with highest Sharpe Ratio
    max_sharpe_port = results_frame.iloc[results_frame['sharpe'].idxmax()]
    #locate positon of portfolio with minimum standard deviation
    min_vol_port = results_frame.iloc[results_frame['stdev'].idxmin()]
 
    #create scatter plot coloured by Sharpe Ratio
    #plt.scatter(results_frame.stdev,results_frame.ret,c=results_frame.sharpe,cmap='RdYlBu')
    #plt.xlabel('Volatility')
    #plt.ylabel('Returns')
    #plt.colorbar()
    #plot red star to highlight position of portfolio with highest Sharpe Ratio
    #plt.scatter(max_sharpe_port[1],max_sharpe_port[0],marker=(5,1,0),color='r',s=1000)
    #plot green star to highlight position of minimum variance portfolio
    #plt.scatter(min_vol_port[1],min_vol_port[0],marker=(5,1,0),color='g',s=1000)
    #plt.show()
    return max_sharpe_port

In [3]:
stocks = ['ALL',
 'MMC',
 'PGR',
 'MA',
 'AIG',
 'AON',
 'CME',
 'CFG',
 'RF',
 'PYPL',
 'ADBE',
 'NVDA',
 'FB',
 'CTXS',
 'XLNX',
 'NTAP',
 'ADSK',
 'LRCX',
 'AYI',
 'AMD',
 'WEC',
 'SCG',
 'CMS',
 'XEL',
 'NEE',
 'EXC',
 'PPL',
 'PCG',
 'NRG',
 'CPN',
 'VLO',
 'EQT',
 'WMB',
 'KMI',
 'CVX',
 'RIG',
 'NFX',
 'SLB',
 'ESV',
 'SWN']

In [4]:
#Brenchmark = ['^GSPC']

ETF = ['SPY', 'EEM', 'OIL', 'FXI', 'JNK', 'VCSH']

In [5]:
max_sharpe_port_stock_pre = max_sharpe_port(stocks,'2017-02-15','2018-02-15')

UnboundLocalError: local variable 'max_sharpe_port' referenced before assignment

In [ ]:
max_sharpe_port_stock_cur = max_sharpe_port(stocks,'2017-02-26','2018-02-26')

In [ ]:
max_sharpe_port_etf_pre = max_sharpe_port(ETF,'2017-02-15','2018-02-15')

In [ ]:
max_sharpe_port_etf_cur = max_sharpe_port(ETF,'2017-02-26','2018-02-26')

In [ ]:
previous_stocks = max_sharpe_port_stock_pre.as_matrix()
current_stocks = max_sharpe_port_stock_cur.as_matrix()
previous_etf = max_sharpe_port_etf_pre.as_matrix()
current_etf = max_sharpe_port_etf_cur.as_matrix()

In [ ]:
stock_weight_change = current_stocks - previous_stocks
etf_weight_change = current_etf - previous_etf

In [ ]:
stock_data_cur = yf.download(stocks,'2018-02-26','2018-02-26')['Adj Close']

In [ ]:
etf_data_cur = yf.download(ETF,'2018-02-26','2018-02-26')['Adj Close']

In [ ]:
#change in dollar amount for each stock
print(np.multiply(stock_data_cur,stock_weight_change[3:]))

In [ ]:
print(np.multiply(etf_data_cur,etf_weight_change[3:]))

In [ ]:
def compute_return(stocks,start,end,max_sharpe_port):
    try:
        dataPrevious = yf.download(stocks,start,start)['Adj Close']
        dataCurrent = yf.download(stocks,end,end)['Adj Close']
    except:
        return compute_return(stocks,start,end,max_sharpe_port)
    previous = dataPrevious.as_matrix()
    Current = dataCurrent.as_matrix()
    
    current_return = (Current - previous)/Current
    weights = max_sharpe_port[3:]
    weighted_return = np.dot((Current - previous)/Current,weights)
    
    return weighted_return
    
    
    
    

In [ ]:
def betaCalculate(stocks):
    result = []
    for i in stocks:
        try:
            ret = yf.download(i,'2017-02-26','2018-02-26')['Adj Close'].pct_change()[1:].values
        except:
            print('error') #betaCalculate(stocks)
        result.append(ret)
    return result

In [ ]:
stockBeta = betaCalculate(stocks)

In [ ]:
benchmarkBeta = betaCalculate(['^GSPC'])
print(benchmarkBeta)

In [ ]:
etfBeta = betaCalculate(ETF)

In [ ]:
import statsmodels.api as sm

In [ ]:
from statsmodels import regression

In [ ]:
def linreg(ret_stocks, ret_benchMark):
    ret_stocks = sm.add_constant(ret_stocks)
    model = regression.linear_model.OLS(ret_benchMark, ret_stocks).fit()
    ret_stocks = ret_stocks[:,1]
    return model.params[0], model.params[1]
#alpha,beta = linreg(etfBeta, benchmarkBeta)

In [ ]:
Aplha = []
Beta = []
for i in etfBeta:
    #print(i.shape)
    alpha,beta = linreg(list(i),benchmarkBeta[0])
    Aplha.append(alpha)
    Beta.append(beta)

In [ ]:
len(benchmarkBeta[0])

In [ ]:
np.dot(Beta,previous_etf[3:])

In [ ]:
np.dot(Aplha,current_etf[3:])

In [ ]:
Aplha = []
Beta = []
for i in stockBeta:
    #print(i.shape)
    alpha,beta = linreg(list(i),benchmarkBeta[0])
    Aplha.append(alpha)
    Beta.append(beta)

In [ ]:
np.dot(Beta,previous_stocks[3:])

In [ ]:
np.dot(Aplha,current_stocks[3:])

In [6]:
current_stocks

NameError: name 'current_stocks' is not defined